# Calculando a pressão de vapor

De acordo com o [artigo do grupo do professo Stephan Grimme](https://pubs.rsc.org/en/content/articlehtml/2022/em/d2em00271j), a pressão de vapor pode ser calculada usando a equação de Ben-Naimm.

\begin{equation} P=\frac{R\times T\times \rho_{liq}}{MW}\times exp\left(\frac{1}{R\times T}\times(\overline{G^*}_{liq}-\overline{G^*}_{vap})\right) \end{equation}
  
| Variável | Descrição | Unidade |
|---|---|---|
| P | pressão de vapor | Pascal (PA) |
| R | constante dos gases | J/(mol*K) |
| T | temperatura | K |
| p_liq | densidade do líquido | kg/m^3 |
| MW | massa molar | kg/mol |
| G_liq | energia livre da fase líquida | J/mol |
| G_vap | energia livre da fase vapor | J/mol |

# Calculando a partir de um csv

O input esperado para o programa é um arquivo no formato CSV com as seguintes colunas.
  
| Nome da coluna | Descrição | Unidade |
|---|---|--|
| Molecula | O nome da molécula | - |
| Comentario | O comentário que você quiser adicionar | - |
| R | constante dos gases | J/(mol*K) |
| T | temperatura | K |
| p_liq | densidade do líquido | kg/m^3 |
| MW | massa molar | kg/mol |
| G_vap | energia livre da fase vapor | Hartree |
| G_liq | energia livre da fase líquida | Hartree |
  
* Os valores nas colunas `G_vap` e `G_liq` estão em Hartree, que é a unidade em que o CENSO retorna, ou seja, não precisa converter antes para J/mol, a conversão é feita durante a execução do programa.
* Por padrão, o nome do arquivo que o programa procura é `values.csv`, mas isso você pode mudar abaixo na variável `input_file`.

O programa irá salvar um arquivo denominado `results.csv`, esse arquivo possui as mesmas colunas que o arquivo de entrada, com adição de mais duas colunas.
  
| Nome da coluna | Descrição | Unidade |
|---|---|--|
| PV | Pressão de vapor | Pascal (PA) |
| log_PV | O log na base 10 da pressão de vapor | - |

#### Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np

#### Definindo as funções

In [23]:
def hartree_to_j_mol(x):
    """
    Convert Hartree to J/mol.
    Args:
        x: value in Hartree
    Returns:
        value in J/mol
    """
    return x * 2625.5 * 1000

def vapor_pressure(row):
    """
    Calculate vapor pressure using the Ben-Naim equation of state.
    Args:
        R: gas constant (in J/(mol*K))
        T: temperature (in K)
        p_liq: liquid density (in kg/m^3)
        MW: molecular weight (in kg/mol)
        G_liq: free energy of liquid (in Hartree)
        G_vap: free energy of vapor (in Hartree)
    Returns:
        vapor pressure (in Pa)
    """

    R = float(row["R"])
    T = float(row["T"])
    p_liq = float(row["p_liq"])
    MW = float(row["MW"])
    G_liq = float(hartree_to_j_mol(row["G_liq"]))
    G_vap = float(hartree_to_j_mol(row["G_vap"]))

    a = (R * T * p_liq) / (MW)
    b = np.exp((G_liq - G_vap) / (R * T))
    return a * b

def run(input_file, output_file):
    """
    Run the calculation of vapor pressure.

    Args:
        input_file: path to the input file
        output_file: path to the output file
    Returns:
        df: dataframe with the calculated vapor pressure
    """

    # Open the file
    df = pd.read_csv(input_file)
    print(f"* Arquivo de entrada {input_file} aberto com sucesso!")

    # Calculate vapor pressure
    df["PV"] = df.apply(vapor_pressure, axis=1)

    # Calculate log of vapor pressure
    df["log_PV"] = np.log10(df["PV"])
    
    print("* Pressão de vapor calculada com sucesso!")

    # Save the file
    df.to_csv(output_file, index=False)
    print(f"* Arquivo de saída {output_file} que contém os resultados salvo com sucesso!")

    print("* Imprimindo as pressões de vapor calculadas!")
    display(df[["Molecula", "Comentario", "PV", "log_PV"]].dropna(how="all"))

    print(f"* obs: no arquivo {output_file} os valores estão salvos com todas as casas decimais.")

    return df.dropna(how="all")

#### Executando o programa

In [24]:
# Define the input and output filename
input_file = "values.csv"
output_file = "results.csv"

if __name__ == "__main__":
    df = run(input_file=input_file, output_file=output_file)

* Arquivo de entrada values.csv aberto com sucesso!
* Pressão de vapor calculada com sucesso!
* Arquivo de saída results.csv que contém os resultados salvo com sucesso!
* Imprimindo as pressões de vapor calculadas!


,Molecula,Comentario,PV,log_PV
0,GLY,NaN,0.016421,-1.784593
1,GLY_corr,NaN,0.019026,-1.720657
2,c10,NaN,300.013385,2.477141
3,c10_corr,NaN,293.103728,2.467021
4,nap,NaN,20.726263,1.316521
5,ABA,replicata 1,0.000009,-5.029158
6,ABA_corr,replicata 1,0.000012,-4.927919
7,ABA,replicata 2,0.000006,-5.227266
8,ABA_corr,replicata 2,0.000006,-5.193597
9,ABA,replicata 3,0.000008,-5.095716


* obs: no arquivo results.csv os valores estão salvos com todas as casas decimais.
